In [31]:
import numpy as np
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
from itertools import permutations
from generator import next_graph

In [46]:
def create_model(model_name, G, tree, first_node_idx = 0):
    with gp.Env(empty = True) as env:
        env.setParam('LogToConsole', 0)
        env.start()
        
        model=gp.Model(model_name, env=env)
        
        n = len(G)
        n_list = list(G)
        n_dict = {n_list[idx]: idx for idx in range(n)}
        first_node = n_list[first_node_idx]

        tree_closure = nx.transitive_closure_dag(tree)
        A = nx.to_numpy_matrix(G, nodelist=list(G))

        x_ij, cost = gp.multidict(dict(np.ndenumerate(A)))
        y_ij, dummy_cost = gp.multidict(dict(np.ndenumerate(np.zeros((n,n)))))

        ### VARIABLES
        x = model.addVars(x_ij, vtype=GRB.BINARY, name='x')
        y = model.addVars(y_ij, name='y')

        ### OBJECTIVE
        objective = model.setObjective(x.prod(cost), GRB.MINIMIZE)

        ### CONSTRAINTS
        ### FLOW CONSERVATION
        for v_i in range(n):
            model.addConstr(sum(x[v_i,v_j] for v_j in range(n) 
                                                if (n_list[v_i],n_list[v_j]) in G.edges) == 1, f'fc_outer_{n_list[v_i]}')

            model.addConstr(sum(x[v_j,v_i] for v_j in range(n) 
                                                if (n_list[v_j],n_list[v_i]) in G.edges) == 1, f'fc_inner_{n_list[v_i]}')
  
    

        ### SUB-TOUR ELIMINATION
        for e in G.edges:
            i = n_dict[e[0]]
            j = n_dict[e[1]]

            if not first_node_idx in (i,j):               
                model.addConstr(y[i,j] - x[i,j] >= 0, f'ste1_{n_list[i]}{n_list[j]}')
                model.addConstr(y[i,j] + x[j,i] == 1, f'ste2_{n_list[i]}{n_list[j]}')

        for e1, e2, e3 in [p for p in permutations([e for e in G.edges 
                                                    if not first_node in e],3) 
                           if sorted(p) == p and p[0][1] == p[1][0] and p[1][1] == p[2][0] and p[2][1] == p[0][0]]:
            i = n_dict[e1[0]]
            j = n_dict[e2[0]]
            k = n_dict[e3[0]]
            model.addConstr(y[i,j] + y[j,k] + y[k,i] <= 2, f'ste3_{n_list[i]}{n_list[j]}{n_list[k]}')

        ### PRECEDENCE CONSTRAINTS

        for e in tree_closure.edges:
            i = n_dict[e[0]]
            j = n_dict[e[1]]
            
            model.addConstr(y[i,j] == 1, f'pc_{n_list[i]}{n_list[j]}')

        return model, x


def optimizeModel(model):
    model.setParam(GRB.Param.TimeLimit,1)
    model.optimize()
    
    return model.status, model.objboundc
    
    


In [44]:
G=nx.DiGraph()
edge_list=[('1','2',10),('1','3',15),('2','3',5), ('3','4',10), ('4','2', 25), ('4','1', 35)]
G.add_weighted_edges_from(edge_list)
A=nx.to_numpy_matrix(G, nodelist=list(G))
len(G)
tree = nx.DiGraph()
tree.add_edges_from([('1','2'),('1','3')])
nd_list = list(G)
{nd_list[idx]: idx for idx in range(len(nd_list))}

{'1': 0, '2': 1, '3': 2, '4': 3}

In [47]:
import timeit
def to_measureTime():
    G, tree, sigma = next_graph()

    model, x = create_model('proba', G, tree)
    model.write('model.lp')
    res = optimizeModel(model)
    print(f'Result: {res}')

timeit.timeit(to_measureTime, number = 10) / 10

Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 836.0)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 825.0)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 790.0)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 724.0)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 779.0)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 688.0)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 791.0)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 694.0)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 794.0)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  

7.548105078099979